In [1]:
import json
import os

import nltk
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize


In [2]:
schema = ["url", "archive", "title", "date", "text", "summary", "compression", "coverage", "density", "compression_bin", "coverage_bin", "density_bin"]

In [7]:
def remove_invalid_lines(file_path):
    valid_lines = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line in lines:
        try:
            json.loads(line)
            valid_lines.append(line)
        except json.JSONDecodeError:
            continue
    
    with open(file_path, 'w') as file:
        file.writelines(valid_lines)

In [8]:
for file_name in os.listdir("data"):
    if file_name.startswith(("test_", "train_", "dev_")):
        print(file_name)
        remove_invalid_lines(os.path.join("data", file_name))

dev_part_aa
dev_part_ab
dev_part_ac
dev_part_ad
dev_part_ae
dev_part_af
test_part_aa
test_part_ab
test_part_ac
test_part_ad
test_part_ae
test_part_af
train_part_aa
train_part_ab
train_part_ac
train_part_ad
train_part_ae
train_part_af
train_part_ag
train_part_ah
train_part_ai
train_part_aj
train_part_ak
train_part_al
train_part_am
train_part_an
train_part_ao
train_part_ap
train_part_aq
train_part_ar
train_part_as
train_part_at
train_part_au
train_part_av
train_part_aw
train_part_ax
train_part_ay
train_part_az
train_part_ba
train_part_bb
train_part_bc
train_part_bd
train_part_be
train_part_bf
train_part_bg
train_part_bh
train_part_bi
train_part_bj
train_part_bk
train_part_bl
train_part_bm
train_part_bn
train_part_bo
train_part_bp
train_part_bq
train_part_br
train_part_bs
train_part_bt
train_part_bu
train_part_bv


In [9]:
records = pd.DataFrame(columns=schema)

for file_name in os.listdir("data"):
    if file_name.startswith(("test_", "train_", "dev_")):
        path = os.path.join("data", file_name)
        
        with open(path, 'r', encoding='utf-8') as file:
            print(file_name)
            jsonObj = pd.read_json(path_or_buf=path, lines=True)
            records = pd.concat([records, jsonObj], ignore_index=True)

data = pd.DataFrame(records)
data.head()

dev_part_aa


C:\Users\ice1s\AppData\Local\Temp\ipykernel_26536\3648510064.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  records = pd.concat([records, jsonObj], ignore_index=True)


dev_part_ab
dev_part_ac
dev_part_ad
dev_part_ae
dev_part_af
test_part_aa
test_part_ab
test_part_ac
test_part_ad
test_part_ae
test_part_af
train_part_aa
train_part_ab
train_part_ac
train_part_ad
train_part_ae
train_part_af
train_part_ag
train_part_ah
train_part_ai
train_part_aj
train_part_ak
train_part_al
train_part_am
train_part_an
train_part_ao
train_part_ap
train_part_aq
train_part_ar
train_part_as
train_part_at
train_part_au
train_part_av
train_part_aw
train_part_ax
train_part_ay
train_part_az
train_part_ba
train_part_bb
train_part_bc
train_part_bd
train_part_be
train_part_bf
train_part_bg
train_part_bh
train_part_bi
train_part_bj
train_part_bk
train_part_bl
train_part_bm
train_part_bn
train_part_bo
train_part_bp
train_part_bq
train_part_br
train_part_bs
train_part_bt
train_part_bu
train_part_bv


,url,archive,title,date,text,summary,compression,coverage,density,compression_bin,coverage_bin,density_bin
0,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,"Despite U.S. Aid, Coca Cultivation On Rise in ...",2033-08-19 18:51:59,"""The prices of oranges, mandarins, coffee and ...","COROICO, Bolivia -- Benito Cocarico admits tha...",11.625000,0.625000,1.000000,low,low,abstractive
1,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,2.5 Million Indians Stranded by Floods,2033-08-19 18:51:59,"NEW DELHI, Sept. 2 -- Nearly 2.5 million India...","World news headlines from the Washington Post,...",13.065217,0.347826,0.347826,low,low,abstractive
2,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,Grassley Seeks Information About Hospital With...,2033-08-19 18:51:59,The ranking Republican on the Senate Finance C...,Latest news on the US federal government. Inf...,26.083333,0.500000,0.555556,medium,low,abstractive
3,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,Gilchrest Breaks With the GOP In Md. 1st Distr...,2033-08-19 18:51:59,"U.S. Rep. Wayne T. Gilchrest, who lost a bitte...","U.S. Rep. Wayne T. Gilchrest, who lost a bitte...",19.475000,1.000000,40.000000,medium,high,extractive
4,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,'Raising the Bar' and Clearing It in the Ratings,2033-08-19 18:51:59,Steven Bochco set a new record Monday when his...,Steven Bochco set a new record Monday when his...,16.555556,1.000000,45.000000,medium,high,extractive


In [10]:
data = pd.DataFrame(records)
data.head()

,url,archive,title,date,text,summary,compression,coverage,density,compression_bin,coverage_bin,density_bin
0,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,"Despite U.S. Aid, Coca Cultivation On Rise in ...",2033-08-19 18:51:59,"""The prices of oranges, mandarins, coffee and ...","COROICO, Bolivia -- Benito Cocarico admits tha...",11.625000,0.625000,1.000000,low,low,abstractive
1,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,2.5 Million Indians Stranded by Floods,2033-08-19 18:51:59,"NEW DELHI, Sept. 2 -- Nearly 2.5 million India...","World news headlines from the Washington Post,...",13.065217,0.347826,0.347826,low,low,abstractive
2,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,Grassley Seeks Information About Hospital With...,2033-08-19 18:51:59,The ranking Republican on the Senate Finance C...,Latest news on the US federal government. Inf...,26.083333,0.500000,0.555556,medium,low,abstractive
3,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,Gilchrest Breaks With the GOP In Md. 1st Distr...,2033-08-19 18:51:59,"U.S. Rep. Wayne T. Gilchrest, who lost a bitte...","U.S. Rep. Wayne T. Gilchrest, who lost a bitte...",19.475000,1.000000,40.000000,medium,high,extractive
4,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2008090319id_/http...,'Raising the Bar' and Clearing It in the Ratings,2033-08-19 18:51:59,Steven Bochco set a new record Monday when his...,Steven Bochco set a new record Monday when his...,16.555556,1.000000,45.000000,medium,high,extractive


In [11]:
data = data[['text', 'summary']]
data

,text,summary
0,"""The prices of oranges, mandarins, coffee and ...","COROICO, Bolivia -- Benito Cocarico admits tha..."
1,"NEW DELHI, Sept. 2 -- Nearly 2.5 million India...","World news headlines from the Washington Post,..."
2,The ranking Republican on the Senate Finance C...,Latest news on the US federal government. Inf...
3,"U.S. Rep. Wayne T. Gilchrest, who lost a bitte...","U.S. Rep. Wayne T. Gilchrest, who lost a bitte..."
4,Steven Bochco set a new record Monday when his...,Steven Bochco set a new record Monday when his...
...,...,...
468775,The Federal Emergency Management Agency made $...,The Federal Emergency Management Agency made $...
468776,"SOWERBY BRIDGE, England -- During the first tw...","World news headlines from the Washington Post,..."
468777,Elizabeth Taylor has White Diamonds. Coco Chan...,"Get Washington DC, Maryland, Virginia news. In..."
468778,"BALTIMORE, May 18 -- A disease believed to be ...","Get sports news, schedules, rosters for Washin..."


In [12]:
data.describe()

,text,summary
count,468780,468780
unique,7794,5270
top,"Each Wednesday (okay, this one is a little lat...",Join live discussions from the Washington Post...
freq,120,37440


In [13]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)
test_data.to_csv('test_data.csv')
val_data.to_csv('val_data.csv')

In [14]:
train_data.drop_duplicates()

,text,summary
313427,"» This Story:Read +|Watch +\n\n{ ""movie"":""http...",The Pentagon Memorial is to be dedicated on Se...
401856,"Sure, there were cathedrals around every corne...","Find Washington DC, Virginia and Maryland trav..."
54855,The actions far inside undisputed Georgian ter...,"OUTSIDE GORI, Georgia, Aug. 13 -- A day after ..."
227165,"The Egyptian bureau of al-Hurra, an Arabic-lan...",CAIRO First of two articles The Egyptian bure...
205776,Among those being forced to move is Sister Ang...,"SANTA BARBARA, Calif. -- In Southern Californi..."
...,...,...
26362,It is rare that a retirement announcement by a...,It is rare that a retirement announcement by a...
313833,If you had to put together the Help Wanted ad ...,A simple and deceptively tricky question: What...
394615,"In the grand narrative of World War II, the Ba...",The AK-47 has become the world's most prolific...
448182,From the opening line of his statement yesterd...,From the opening line of his statement yesterd...


In [15]:
text_len = train_data['text'].apply(len)
summary_len = train_data['summary'].apply(len)

print(f"Text:\n min: {text_len.min()}\n max: {text_len.max()}\n mean: {text_len.mean()}")
print(f"Summary:\n min: {summary_len.min()}\n max: {summary_len.max()}\n mean: {summary_len.mean()}")


Text:
 min: 603
 max: 592186
 mean: 8733.952616281653
Summary:
 min: 3
 max: 873
 mean: 205.7146103501708


In [16]:
train_data = train_data[train_data['summary'].apply(len) <= 200]
train_data

,text,summary
402312,Chris Clark didn't hesitate when the question ...,Chris Clark scores two goals as the Capitals w...
135975,Small Internet radio stations were offered a b...,Small Internet radio stations were offered a b...
79797,"Babbin is the author of three books, most rece...",Former Deputy Undersecretary of Defense Jed L....
243965,"One of the many unique things about Farid, the...",Islam's Advance on PostGlobal; blog of politic...
371316,If I told you that there was something in Wash...,The urgent issue of war and peace in Iraq shou...
...,...,...
354119,Al Gore became the third Baptist to win the No...,A conversation on religion with Jon Meacham an...
461378,Erica Williams worries about the wave of polit...,Erica Williams worries about the wave of polit...
15260,I'm always trying to crack the credit-scoring ...,I'm always trying to crack the credit-scoring ...
260263,President Bush is so predictable! He needs a w...,Miriam Leitao at PostGlobal on PostGlobal; blo...


In [17]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ice1s\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ice1s\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [ ]:
train_data_tokens = pd.DataFrame()
train_data_tokens['text'] = train_data['text'].apply(word_tokenize)
train_data_tokens['summary'] = train_data['summary'].apply(word_tokenize)
train_data_tokens

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/mika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
stop_words = set(stopwords.words('english'))

In [20]:
train_data_stopwords = pd.DataFrame()

train_data_stopwords['text'] = train_data_tokens['text'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])
train_data_stopwords['summary'] = train_data_tokens['summary'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

train_data_stopwords

,text,summary
602897,"[Earth, Day, ,, safe, start, ragging, Thoreau,...","[Thoreau, environmental, iconoclast, champion,..."
56238,"[Curtiss, Anderson, ,, edited, Ladies, Home, J...","[Hearst, Magazines, ,, Mr., Anderson, helped, ..."
361665,"[politics, issues, liberals, conservatives, sh...","[n't, argument, law, 's, real-world, effects, ..."
190568,"[Thirty, years, ago, Air, India, plane, blown,...","[flight, Canada, India, exploded, 1985, ,, 329..."
113312,"[John, Elway, opened, door, speculation, Tim, ...","[John, Elway, says, Broncos, considering, movi..."
...,...,...
580538,"[federal, judge, ruled, Congress, violated, Co...","[federal, judge, ruled, Congress, violated, Co..."
397314,"[dramas, seem, tailor-made, Internet, 's, ephe...","[dramas, seem, tailor-made, Internet, 's, ephe..."
14996,"[Every, often, smartest, streets, squares, cen...","[Grade-II, listed, London, townhouse, boasts, ..."
515240,"[five, justices, Supreme, Court, disabled, Vot...","[Supreme, Court, ruling, ,, country, must, rai..."


In [21]:
lemmatizer = WordNetLemmatizer()

In [22]:
train_data_lemmatize = pd.DataFrame()
nltk.download('wordnet')

train_data_lemmatize['text'] = train_data_stopwords['text'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
train_data_lemmatize['summary'] = train_data_stopwords['summary'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

train_data_lemmatize


[nltk_data] Downloading package wordnet to /Users/mika/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,summary
602897,"[Earth, Day, ,, safe, start, ragging, Thoreau,...","[Thoreau, environmental, iconoclast, champion,..."
56238,"[Curtiss, Anderson, ,, edited, Ladies, Home, J...","[Hearst, Magazines, ,, Mr., Anderson, helped, ..."
361665,"[politics, issue, liberal, conservative, share...","[n't, argument, law, 's, real-world, effect, ,..."
190568,"[Thirty, year, ago, Air, India, plane, blown, ...","[flight, Canada, India, exploded, 1985, ,, 329..."
113312,"[John, Elway, opened, door, speculation, Tim, ...","[John, Elway, say, Broncos, considering, movin..."
...,...,...
580538,"[federal, judge, ruled, Congress, violated, Co...","[federal, judge, ruled, Congress, violated, Co..."
397314,"[drama, seem, tailor-made, Internet, 's, ephem...","[drama, seem, tailor-made, Internet, 's, ephem..."
14996,"[Every, often, smartest, street, square, centr...","[Grade-II, listed, London, townhouse, boast, e..."
515240,"[five, justice, Supreme, Court, disabled, Voti...","[Supreme, Court, ruling, ,, country, must, rai..."


In [23]:
train_data_lemmatize.to_csv('train_data.csv')